# 1. 크롤링

In [1]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
import time
import pandas as pd
import re
import pickle

In [8]:
service = Service('C:/Users/khb16/Desktop/code/PROJECT/Recommandation/chromedriver.exe')
driver = webdriver.Chrome(service = service)

In [114]:
pages = [] # 크롤링 오류 확인 위해 page 수 기록
title = []
lyric = []
artist = []
genres = []
writer = []
rank = []
p = re.compile("'(\d+)'")
    
for page in range(14651,15001,50): # 1페이지만 추출(1,51,50), 발라드 gnrCode=GN0100, 댄스 gnrCode=GN0200
    url = "https://www.melon.com/genre/song_list.htm?gnrCode=GN0200#params%5BgnrCode%5D=GN0200&params%5BdtlGnrCode%5D\
                =&params%5BorderBy%5D=POP&params%5BsteadyYn%5D=N&po=pageObj&startIndex={}".format(page) # 댄스 페이지
    driver.get(url)
    time.sleep(2)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    
    # 한 페이지에 있는 50개의 songId 추출
    songid = [] # songid

    for i in range(50):
        data = soup.find_all("a", class_= "btn button_icons type03 song_info")[i].get('href')
        id_ = p.search(str(data)).group(1)
        songid.append(id_)
        f = soup.select('td span.rank')[i].text
        rank.append(f)

    # 해당 songId 입력 후 정보 추출
    for song in songid:
        lyric_url = 'https://www.melon.com/song/detail.htm?songId={}'.format(song)
        driver.get(lyric_url)
        time.sleep(2)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        a = soup.select('div.song_name')[0].text[3:].strip() # 제목
        title.append(a)
        try:
            driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
            ly = driver.find_element_by_css_selector('.lyric.on')
            b = ly.text
            lyric.append(b)
        except:
            lyric.append('가사없음')
        ar = driver.find_element_by_css_selector(".artist")
        c = ar.text
        artist.append(c)
        d = soup.select('dl.list dd')[2].text # 장르
        genres.append(d)
        try:
            e = soup.select('div.section_prdcr a.artist_name')[0].text # 작사가 1명
            writer.append(e)
        except:
            writer.append('작사가없음')
        pages.append(page)

<ipython-input-114-c646142e86c3>:39: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_css_selector(".button_more.arrow_d").click() # 가사
<ipython-input-114-c646142e86c3>:40: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ly = driver.find_element_by_css_selector('.lyric.on')
<ipython-input-114-c646142e86c3>:45: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  ar = driver.find_element_by_css_selector(".artist")


IndexError: list index out of range

In [115]:
col = ['pages','rank','title','lyric','artist','genres','writer']
songlist = pd.DataFrame([pages, rank,title, lyric, artist, genres, writer]).T
songlist.columns = col
songlist

,pages,rank,title,lyric,artist,genres,writer
0,14301,14301,Timeless (Space Mix),어쩜 살아가다보면 한 번은\n날 찾을지 몰라\n난 그 기대 하나로 오늘도\n힘겹게 ...,SG 워너비,댄스,작사가없음
1,14301,14302,Red Sun (Feat. Brian of FlytotheSky),(자 이제부터 오랜 기억\n속으로 돌아가 봅니다)\n어머니 난 그녀가 하지 말라는 ...,장우혁,댄스,이경
2,14301,14303,Boy Is Mine,ha ha ye come on back to you\nye check it out ...,코요태,댄스,그레이 (Gray)
3,14301,14304,나 어떡해,나 어떻게 해 구해줘 빨리 Honey\n지금 난 너를 원해 후후후\n나 어떻게 해 ...,엔알지 (NRG),댄스,천명훈
4,14301,14305,사랑하는 사람아,서로가 모르는\n사람들처럼\n한 마디 인사없이\n헤어지나요\n지워야 할 그 사람\n...,박남정,댄스,작사가없음
...,...,...,...,...,...,...,...
395,14651,14696,짠짜라바운스,안녕하세요 나의 사랑\n너의 사랑 김경진이 올시다\n여러분 신나게 흔들고 싶으시죠\...,디제이브라더,댄스,사우스하이 (South High)
396,14651,14697,FEEDBACK (너의 대답은),Think about\nThink about you\n어떤 느낌이 찾아와\n아무것도...,레이디스 코드,댄스,원택 (1Take)
397,14651,14698,Love Trip(모트라인 브랜드송 VER.1),부릉부릉 그대 맘에 시동 걸어\n뚜뚜 신호음 자꾸 들려와\n또또 읽지도 못한 메세지...,피치베리,댄스,등푸른
398,None,14699,None,None,None,None,None


In [116]:
songlist = songlist.head(350)
songlist

,pages,rank,title,lyric,artist,genres,writer
0,14301,14301,Timeless (Space Mix),어쩜 살아가다보면 한 번은\n날 찾을지 몰라\n난 그 기대 하나로 오늘도\n힘겹게 ...,SG 워너비,댄스,작사가없음
1,14301,14302,Red Sun (Feat. Brian of FlytotheSky),(자 이제부터 오랜 기억\n속으로 돌아가 봅니다)\n어머니 난 그녀가 하지 말라는 ...,장우혁,댄스,이경
2,14301,14303,Boy Is Mine,ha ha ye come on back to you\nye check it out ...,코요태,댄스,그레이 (Gray)
3,14301,14304,나 어떡해,나 어떻게 해 구해줘 빨리 Honey\n지금 난 너를 원해 후후후\n나 어떻게 해 ...,엔알지 (NRG),댄스,천명훈
4,14301,14305,사랑하는 사람아,서로가 모르는\n사람들처럼\n한 마디 인사없이\n헤어지나요\n지워야 할 그 사람\n...,박남정,댄스,작사가없음
...,...,...,...,...,...,...,...
345,14601,14646,향 (Scentist) (Inst.),붉은 꽃잎 한 장 어린잎도 한 장\n환상의 수치화 곤두세운 촉각\n얼음 같은 달빛 ...,VIXX (빅스),댄스,Jake K (ARTiffect)
346,14601,14647,비상,Dream’s like an whiskey\n너무 독하고 달지\nI don’t wa...,크로스진 (CROSS GENE),댄스,ZNEE
347,14601,14648,내일도 맑음,everyday 그대와 함께라면\n내일도 맑음\neverytime 우리 둘 함께라면...,금조,"댄스, 국내드라마",알고보니 혼수상태
348,14601,14649,Let Me Stay With You,너는 매일 나를 보면 웃어주고\n너는 매일 나랑 장난도 잘 치지\n너는 내게 너의 ...,더 이스트라이트 (TheEastLight.),댄스,Midas-T


In [118]:
songlist = songlist.tail(300)

In [119]:
songlist.to_pickle('dance_14351-14650.pkl', protocol = 4) # 발라드는 ballad로 저장

In [219]:
songlist.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   pages   100 non-null    object
 1   title   100 non-null    object
 2   lyric   100 non-null    object
 3   artist  100 non-null    object
 4   genres  100 non-null    object
 5   writer  100 non-null    object
dtypes: object(6)
memory usage: 4.8+ KB
